In [1]:
%tensorflow_version 1.x
!apt-get update
!pip install pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg

TensorFlow 1.x selected.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [898 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backpor

Import essentials and set up display.


In [2]:
import numpy as np
import tensorflow as tf
import gym, copy, math, cv2
from gym.wrappers import Monitor
from skimage.color import rgb2gray
from scipy.spatial import cKDTree

import matplotlib.pyplot as plt
import glob, io, base64
from IPython.display import HTML
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

Load distance model.

In [3]:
from tensorflow.keras.models import load_model
from google.colab import drive
drive.mount('/content/drive')
filepath = "/content/drive/My Drive/distance.h5"
model = load_model(filepath)

def predict(a, b):
    a = np.transpose(a, (2, 0, 1))
    b = np.transpose(b, (2, 0, 1))
    data = np.reshape(np.concatenate((a, b), axis = 0), (-1, 4, ROWS, COLS))
    
    pre = model.predict(data)
    return pre[0][0]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Parameters.

In [4]:
MAX_EPISODE = 5000
MAX_EP_STEPS = 500 
RENDER = False
LR_A = 0.0005 
ROWS = 160
COLS = 240
state_size = (1, ROWS, COLS, 4)
COLS2 = 20
ROWS2 = 20

Helper functions.

In [5]:
def crop_img(img):
    shape = (COLS2, ROWS2)
    pixels = img.flatten()[::-1] 
    ylen = img.shape[1]
    flag = False
    idx1 = idx2 = None
    for idx in range(len(pixels)):
        if pixels[idx] == 1:
            if not flag:
                idx1 = idx
            flag = True
        if flag and pixels[idx] == 0:
            idx2 = idx
            break
    middle = int((idx2 + idx1) / 2)
    x = int((len(pixels) - middle) / img.shape[1])
    y = (len(pixels) - middle) % img.shape[1]
    starty = max(0, y - 40)
    startx = max(0, x - 145)
    img = np.array(img[startx:startx+80, starty:starty+80], dtype=np.float32)
    img = np.array(np.array(cv2.resize(img, (shape[0], shape[1])), dtype = bool), dtype=int)
    return np.reshape(img, (ROWS2, COLS2, 1))

def adaptive_lr(solve, lr_episode, t, flag):
    if t >= 350 and not flag:
        flag = True
    if solve == 2: return max(501, lr_episode), flag
    if solve == 3: return max(701, lr_episode), flag
    if solve == 5: return max(901, lr_episode), flag
    return lr_episode, flag

Actor class.

In [6]:
class Actor(object):
    def __init__(self, sess, n_features, n_actions, lr=0.001):
        self.sess = sess
        self.s = tf.placeholder(tf.float32, [1, ROWS2, COLS2, 4], "state")
        self.a = tf.placeholder(tf.int32, (), "act")
        self.td_error = tf.placeholder(tf.float32, (), "td_error") 
        self.TAU = 0.6
        self.p = 1
        self.changeflag = True

        with tf.variable_scope('Actor'):
            filter = tf.Variable(tf.random_normal([5, 5, 1, 16], mean = 0.0, stddev = 0.01))
            l1 = tf.nn.conv2d(self.s, filter, strides = 2, padding = "SAME")
            l1 = tf.layers.flatten(l1)

            self.l1 = tf.layers.dense(inputs = l1, units = 20, activation = None, kernel_initializer = tf.random_normal_initializer(0., .01),
                bias_initializer = tf.constant_initializer(0.01),  name = 'l1')

            self.acts_prob = tf.layers.dense(inputs = self.l1, units = n_actions, activation = tf.nn.softmax, kernel_initializer = tf.random_normal_initializer(0., .01),  # weights
                bias_initializer = tf.constant_initializer(0.01), name = 'acts_prob')

        with tf.variable_scope('logprob'):
            log_prob = tf.log(self.acts_prob[0, self.a])
            self.exp_v = tf.reduce_mean(log_prob) 
            
        with tf.variable_scope('train'):
            self.global_step = tf.Variable(0, trainable = False)
            self.learning_rate = tf.train.exponential_decay(LR_A, self.global_step, 100, 0.5, staircase = True)
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
            self.gvs = self.optimizer.compute_gradients(-self.exp_v)

            def multtd(grad):
                if grad is None:
                    return grad
                return self.td_error * grad

            self.new_gvs = [(multtd(grad), var) for grad, var in self.gvs]
            self.train_op = self.optimizer.apply_gradients(self.new_gvs)

        with tf.variable_scope('Target'):
            tfilter = tf.Variable(tf.random_normal([5, 5, 1, 16], mean = 0.0, stddev = 0.01))
            tl1 = tf.nn.conv2d(self.s, tfilter, strides = 2, padding = "SAME")
            tl1 = tf.layers.flatten(tl1)
            self.tl1 = tf.layers.dense(inputs = l1, units = 20, activation = None, kernel_initializer = tf.random_normal_initializer(0., .01),
                bias_initializer = tf.constant_initializer(0.01), name = 'l1')
            self.tacts_prob = tf.layers.dense(inputs = self.l1, units = n_actions, activation = tf.nn.softmax, kernel_initializer = tf.random_normal_initializer(0., .01),  # weights
                bias_initializer = tf.constant_initializer(0.01), name = 'acts_prob')

    def learn(self, s, a, td, run):
        s = np.reshape(s, (1, ROWS2, COLS2, 4))
        feed_dict = {self.s: s, self.a: a, self.td_error: td, self.global_step:run}
        _, exp_v, ap, l1= self.sess.run([self.train_op, self.exp_v, self.acts_prob, self.l1], feed_dict)
       
        if run == 120 and self.changeflag:
            self.p = 0
            self.changeflag = False
        return exp_v

    def choose_action(self, s, hardmax=False):
        s = np.reshape(s, (1, ROWS2, COLS2, 4))
        probs = self.sess.run([self.tacts_prob], {self.s: s})[0]
        
        if hardmax:
            return np.argmax(probs[0])
        return np.random.choice(np.arange(probs.shape[1]), p = probs.ravel())

    def init_target(self):
        actor_var_names = tf.trainable_variables("Actor")
        actor_var = self.sess.run(actor_var_names)
        target_var_names = tf.trainable_variables("Target")
        for i, j in zip(target_var_names, actor_var):
            self.sess.run(i.assign(j))

    def update_var(self):
        actor_var_names = tf.trainable_variables("Actor")
        actor_var = self.sess.run(actor_var_names)
        target_var_names = tf.trainable_variables("Target")
        target_var = self.sess.run(target_var_names)
        for a, i, j, k in zip(actor_var_names, target_var_names, target_var, actor_var):
            self.sess.run(i.assign(self.TAU * j + (1 - self.TAU) * k))
            if self.p != 0:
                self.sess.run(a.assign(k + self.p * np.random.normal(0, 5e-3 ,size = k.shape)))

Critic class.

In [7]:
class NNCritic:
    def __init__(self, observation_space, action_space, actor):
        self.action_space = action_space
        self.observation_space = observation_space
        
        # parameters to tune
        self.H = 12
        self.L = 0.5
        self.discount = 0.99

        self.max_size = 500000
        self.ptr = int(0)
        self.next_states, self.next_states2, self.next_states_full = [], [], []
        self.states_all, self.states_full = [], []
        self.rewards = []

        self.tree = None
        self.actor = actor
        self.weights =  np.array([1 for i in range(self.observation_space)] + [1])

    def remember(self, state, action, reward, next_state, step, terminal, img, img_next, s_f, s_f_):
        if len(self.states_all) == self.max_size:
            self.states_all[self.ptr] = np.concatenate((state, [action]))
            self.states_full[self.ptr] = s_f
            self.next_states[self.ptr] = next_state
            self.next_states2[self.ptr] = img_next.flatten()
            self.next_states_full[self.ptr] = s_f_
            self.rewards[self.ptr] = reward
            self.ptr = int((self.ptr + 1) % self.max_size)
        else:
            self.states_all.append(np.concatenate((state, [action])))
            self.states_full.append(s_f)
            self.next_states.append(next_state)
            self.next_states2.append(img_next.flatten())
            self.next_states_full.append(s_f_)
            self.rewards.append(reward)
            
    def act(self, step, state, img, sf):
        if step == self.H:
            return state, 0

        img_ = np.concatenate((np.reshape(img.flatten()[:ROWS2 * COLS2], (ROWS2, COLS2, 1)), np.reshape(img.flatten()[ROWS2 * COLS2:2 * ROWS2 * COLS2], (ROWS2, COLS2, 1)), np.reshape(img.flatten()[2 * ROWS2 * COLS2:3 * ROWS2 * COLS2], (ROWS2, COLS2, 1)), np.reshape(img.flatten()[3 * ROWS2 * COLS2:], (ROWS2, COLS2, 1))), axis = 2)
        a = self.actor.choose_action(img_)

        dd, ii = self.tree.query(self.weights * np.concatenate((state, [a])), k = 1, n_jobs = -1)
        nearest = self.states_all[ii]
  
        dd = predict(self.states_full[ii], sf) if dd !=0 else 0
        
        if self.rewards[ii] == -1:
            return nearest[:-1], -1 + self.L * dd
        else:
            q = self.rewards[ii] + self.discount * self.act(step + 1, self.next_states[ii], self.next_states2[ii], self.next_states_full[ii])[1] + self.L * dd
            return nearest[:-1], q
        
    def learn(self, s, r, s_, run, step, img, img_next, sf, sf_, flag):
        if run == 0: return 0
        if not flag:
            self.tree = cKDTree(self.weights * self.states_all)
        
        n, v = self.act(0, s, img.flatten(), sf)
        n_, v_ = self.act(1, s_, img_next.flatten(), sf_)
        td_error = r + self.discount * v_ - v

        return td_error

Run experiment.

In [ ]:
seed = 1
np.random.seed(seed)
tf.set_random_seed(seed)

env = wrap_env(gym.make('CartPole-v1'))
env = env.unwrapped
env.seed(seed)
env._max_episode_steps = 500
N_F = env.observation_space.shape[0]
N_A = env.action_space.n

sess = tf.Session()
actor = Actor(sess, n_features = 4 * ROWS2 * COLS2, n_actions = N_A, lr = LR_A)
critic = NNCritic(N_F, N_A, actor)   
sess.run(tf.global_variables_initializer())
actor.init_target()

scores = []
flag = False
timesteps, running_reward = 0, 0
i_episode, lr_episode = 0, 0
solve = 0
while i_episode < MAX_EPISODE:
    timesteps += 1
    s = env.reset()
    t = 0
    track_r = []

    img_ = np.array(np.array((env.render(mode = 'rgb_array')[:,:,0]) - 255, dtype = bool), dtype=np.float32)
    img_prev1 = img_prev2 = img_prev = img = crop_img(img_)
    img_prev_full = img_full = np.reshape(cv2.resize(img_, (COLS, ROWS)), (ROWS, COLS, 1))

    while True:
        if RENDER: env.render()
        
        a = actor.choose_action(np.concatenate((img_prev1, img_prev2, img_prev, img), axis = 2))
        s_, r, done, info = env.step(a)
        if done: r = -1

        img_next_ = np.array(np.array((env.render(mode = 'rgb_array')[:,:,0]) - 255, dtype = bool), dtype = np.float32)
        img_next = crop_img(img_next_)
        img_next_full = np.reshape(cv2.resize(img_next_, (COLS, ROWS)), (ROWS, COLS, 1))
     
        critic.remember(s, a, r, s_, t, done, np.concatenate((img_prev1, img_prev2, img_prev, img)), np.concatenate((img_prev2, img_prev,img, img_next)), np.concatenate((img_prev_full, img_full), axis = 2), np.concatenate((img_full, img_next_full), axis = 2))
        track_r.append(r)

        td_error = np.clip(critic.learn(s, r, s_, i_episode, t, np.concatenate((img_prev1, img_prev2, img_prev, img)), np.concatenate((img_prev2, img_prev,img, img_next)), np.concatenate((img_prev_full, img_full), axis = 2), np.concatenate((img_full, img_next_full), axis = 2), flag), -3, 3)

        actor.learn(np.concatenate((img_prev1, img_prev2, img_prev, img), axis = 2), a, td_error, lr_episode) 

        img_prev1 = img_prev2
        img_prev2 = img_prev
        img_prev = img
        img = img_next

        img_prev_full = img_full
        img_full = img_next_full
        s = s_

        t += 1
        timesteps += 1

        if done or t >= MAX_EP_STEPS:
            if t == MAX_EP_STEPS: solve += 1
            lr_episode, flag = adaptive_lr(solve, lr_episode, t, flag)
            
            if flag:
                critic.tree = cKDTree(critic.weights * critic.states_all)

            if i_episode % 2 == 0:
                actor.update_var()
            
            ep_rs_sum = sum(track_r)
            running_reward = running_reward * 0.8 + ep_rs_sum * 0.2
            print("Episode: ", i_episode, " running reward: ", running_reward, " episode reward: ", ep_rs_sum)
            scores.append(ep_rs_sum)
            i_episode += 1
            lr_episode += 1
            break


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Episode:  0  running reward:  2.2  episode reward:  11.0
Episode:  1  running reward:  5.16  episode reward:  17.0
Episode:  2  running reward:  5.928  episode reward:  9.0
Episode:  3  running reward:  9.1424  episode reward:  22.0
Episode:  4  running reward:  9.31392  episode reward:  10.0
Episode:  5  running reward:  10.051136  episode reward:  13.0
Episode:  6  running reward:  11.4409088  episode reward:  17.0
Episode:  7  running reward:  12.952727040000001  episode reward:  19.0
Episode:  8  running reward:  13.162181632000003  episode reward:  14.0
Episode:  9  running reward:  13.129745305600002  episode reward:  13.0
Episode:  10  running reward:  15.503796244480002  episode reward:  25.0
Episode:  11  running reward:  16.403036995584003  episode reward:  20.0
Episode:  12  running rewa